In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from skimage.feature import local_binary_pattern


dataset_path='Dataset'
RANDOMSEED=1277
num_type=15
label_list = [
    "Agriculture", "Airport", "Beach", "City", "Desert", "Forest",
    "Grassland", "Highway", "Lake", "Mountain", "Parking", "Port",
    "Railway", "Residential", "River"
]

In [2]:
code = pd.read_pickle("D:/temp/index_df_code.pkl")
lbp_features = pd.read_pickle("D:/temp/index_df_lbp_features.pkl")

index_df = pd.concat([code, lbp_features], axis=1)


In [3]:
totsize=len(index_df)
print(totsize)
train_size=int(totsize*0.8)
test_size=int(totsize*0.2)

train_per_code=train_size//num_type
test_per_code=test_size//num_type

train_df=pd.DataFrame()
test_df=pd.DataFrame()
# use train_test_split to split test and train set
for code in index_df['code'].unique():
    # every word, each word is code
    sample_code= index_df[index_df['code'] == code]
    train_samples, test_samples = train_test_split(
        sample_code, 
        test_size=test_per_code, 
        train_size=train_per_code,
        random_state=RANDOMSEED, 
        shuffle=True
    )
 
    train_df = pd.concat([train_df, train_samples], ignore_index=True)
    test_df = pd.concat([test_df, test_samples], ignore_index=True)

print('finish')

12000
finish


In [4]:
def printresult_pro(y_true, y_pred, y_proba, label_names=None, top_n=3):
    print("Evaluation Metrics:")
    print("Accuracy Score:", metrics.accuracy_score(y_true, y_pred))
    print("Recall Score (macro):", metrics.recall_score(y_true, y_pred, average='macro'))
    print("F1 Score (macro):", metrics.f1_score(y_true, y_pred, average='macro'))
    print("Confusion Matrix:")
    print(metrics.confusion_matrix(y_true, y_pred))

    # 2. precision / recall / f1
    print("\nPer-Class Performance:")
    report = metrics.classification_report(y_true, y_pred, target_names=label_names, digits=3)
    print(report)

In [5]:
train_lbp = np.array(train_df['lbp_features'].tolist())
test_lbp = np.array(test_df['lbp_features'].tolist())


# pca = PCA(n_components=40)  #
# train_lbp = pca.fit_transform(train_lbp)
# test_lbp = pca.transform(test_lbp)

# scaler = StandardScaler() # scaler
# train_lbp = scaler.fit_transform(train_lbp)
# test_lbp = scaler.transform(test_lbp)

y_train = np.array(train_df['code'])
y_test = np.array(test_df['code'])


KNN-lbp

In [6]:
knn_lbp = KNeighborsClassifier(n_neighbors=3, weights='distance')

knn_lbp.fit(train_lbp, y_train)
knn_lbp_predict_y = knn_lbp.predict(test_lbp)
knn_clf_predict_y_proba = knn_lbp.predict_proba(test_lbp)



C:\Users\lele1\anaconda3\envs\comp9417\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 系统找不到指定的文件。
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\subprocess.py", line 550, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\subprocess.py", line 1028, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\subprocess.py", line 1540, in _execute_child


In [7]:
###### result for KNN-sift_features
print("KNN-lbp:")
#printresult(y_test, knn_lbp_predict_y)
printresult_pro(y_test, knn_lbp_predict_y, knn_clf_predict_y_proba,label_names=label_list)


KNN-lbp:
Evaluation Metrics:
Accuracy Score: 0.49291666666666667
Recall Score (macro): 0.49291666666666667
F1 Score (macro): 0.49090442081383984
Confusion Matrix:
[[ 71  22  17   1  10   0   7   3   2   5   5   2   2   3  10]
 [ 13  59   2  10   2   0   1  20   3  12   6   0  14  11   7]
 [ 10   5  55   1  19   0   5   1  19   6   4  13   0   8  14]
 [  3  11   1  71   0   0   0   9   0   7  13   3  26  10   6]
 [  7   2  18   1  83   1  13   1   3  14   1   1   0   7   8]
 [  0   0   0   1   3 126  11   0   4   7   0   1   0   7   0]
 [  2   1   8   0   9  24 106   2   1   2   0   0   0   2   3]
 [  8  22   2   8   5   0   1  58   2   4   7   3  19  17   4]
 [  2   2  21   2   3   1   1   1  68   6   0  25   0   6  22]
 [  6   9   3   4   7   8   4   0   3  98   0   1   3   4  10]
 [  5  10   8  15   1   0   1  12   2   4  71   7   4  13   7]
 [  3   2   6   5   0   0   0   1  15   0   8 112   0   2   6]
 [  1  17   0  34   1   0   0  19   1   7   4   0  63  10   3]
 [  0   5   4  10 

SVC-LBP

In [8]:
svc_lbp = SVC(kernel='rbf', random_state=RANDOMSEED,probability=True)
svc_lbp.fit(train_lbp, y_train)
svc_lbp_predict = svc_lbp.predict(test_lbp)
svc_lbp_prob = svc_lbp.predict_proba(test_lbp)


In [9]:
#result for svc-lbp
print("svc-lbp:")
printresult_pro(y_test, svc_lbp_predict, svc_lbp_prob,label_names=label_list)


svc-lbp:
Evaluation Metrics:
Accuracy Score: 0.46541666666666665
Recall Score (macro): 0.46541666666666653
F1 Score (macro): 0.45957031713964486
Confusion Matrix:
[[ 75  19  11   0  15   0   2   4   2   8   5   2   4   1  12]
 [ 17  44   1   8   2   1   1  21   0  22  13   0  16   6   8]
 [  5   2  51   1  24   2   1   1  13   8   7  11   0  13  21]
 [  2  19   0  53   2   0   0  14   1  13  13   0  20  18   5]
 [  6   2  19   1  88   1   7   0   0  28   0   0   1   6   1]
 [  0   0   0   0   0 135   7   0   0  15   0   2   0   0   1]
 [  5   0   3   0  14  37  91   1   0   2   1   0   0   4   2]
 [  8  23   1   5   3   3   0  55   0  14   7   2  18  20   1]
 [  1   0  13   0   9   4   3   0  62  13   0  31   0   0  24]
 [  0   3   0   3   7  11   3   0   0 121   0   0   3   6   3]
 [  4  18   0  10   0   0   1  10   0   7  63   6   4  28   9]
 [  5   2   7   2   1   0   0   0  12   2   6 113   0   2   8]
 [  1  13   0  29   0   0   0  20   0  20   8   1  55  11   2]
 [  0   4   0   3 

In [10]:
rf_lbp = RandomForestClassifier(n_estimators=100, random_state=RANDOMSEED)
rf_lbp.fit(train_lbp, y_train)
rf_lbp_predict = rf_lbp.predict(test_lbp)
rf_lbp_prob = rf_lbp.predict_proba(test_lbp)

In [11]:
#result for rf-lbp
print("rf-lbp:")
printresult_pro(y_test, rf_lbp_predict, rf_lbp_prob,label_names=label_list)


rf-lbp:
Evaluation Metrics:
Accuracy Score: 0.5620833333333334
Recall Score (macro): 0.5620833333333334
F1 Score (macro): 0.5599111210579142
Confusion Matrix:
[[ 93  15   9   1  13   0   3   3   2   2   4   1   6   1   7]
 [ 14  67   1  10   1   0   0  21   2   8  11   0  13   9   3]
 [  4   5  69   1  17   2   1   0  15   6   5  10   0   8  17]
 [  2  10   0  80   1   0   0   9   2   5  17   0  23   8   3]
 [ 10   1  15   0  95   0   7   2   1  15   0   0   2   6   6]
 [  0   0   1   0   1 134  11   0   3   6   0   1   0   2   1]
 [  3   1   5   0  11  21 111   4   0   2   0   0   0   1   1]
 [  7  28   1   4   2   0   1  71   1   6   9   1  11  16   2]
 [  1   2  21   0   6   1   2   1  79   7   0  17   0   1  22]
 [  0   2   0   3   8   6   4   0   1 117   0   0   4   9   6]
 [  3  14   1  16   1   0   0   6   0   1  88   7   5  11   7]
 [  4   0   5   6   0   0   0   0  13   1   7 116   0   2   6]
 [  0  15   0  29   0   0   0  20   1   8   4   0  75   5   3]
 [  1   5   0   5   3 